In [ ]:
import psycopg2
import os

In [ ]:
postgres_uri = os.environ.get('URI')

In [ ]:
def execute_sql(sql):
    try:
        conn = psycopg2.connect(postgres_uri)
        cur = conn.cursor()
        cur.execute(sql)
        conn.commit()
    finally:
        if conn is not None:
            conn.close()

In [ ]:
def execute_sql_results(sql):
    try:
        conn = psycopg2.connect(postgres_uri)
        cur = conn.cursor()
        cur.execute(sql)
        rows = cur.fetchall()
        return rows
    finally:
        if conn is not None:
            conn.close()

In [ ]:
sql = 'select film_id, title, description from film limit 10'

In [ ]:
rows = execute_sql_results(sql)
for row in rows:
    print(row)

In [ ]:
sql = """
    select film_id, title, description, 1 - (aidb.encode_text('bert', description)::vector <=>
	aidb.encode_text('bert', 'A movie about cats in Australia')::vector) as cosine_similarity_score from film order by 4 desc limit 10
    """

In [ ]:
rows = execute_sql_results(sql)
for row in rows:
    print(row)

In [ ]:
#inference_server = "https://deepseek7b.samouelian-edb-ai.svc.cluster.local/embeddings"
sql = """
    select aidb.create_model(
    'deepseek',
    'embeddings',
    '{"model":"deepseek-ai/DeepSeek-R1-Distill-Llama-8B", "url":"https://deepseek7b.samouelian-edb-ai.svc.cluster.local/embeddings", 
    "dimensions":7168}'::JSONB);
    """

In [ ]:
execute_sql(sql)

In [ ]:
sql = """
    select film_id, title, description, 1 - (aidb.encode_text('bert', description)::vector <=>
	aidb.encode_text('bert', 'A movie about cats in Australia')::vector) as cosine_similarity_score from film order by 4 desc limit 10
    """

In [ ]:
#Below is a curl REST example for retrieving an embedding

In [9]:
import pycurl
import json
from io import BytesIO

endpoint = "https://grit-llm.samouelian-edb-ai.svc.cluster.local/v1/embeddings"

data = {
     "model": "grit-llm",
     "input": ["this is a test"],
     "encoding_format":"float"
}

post_data = json.dumps(data)
headers = ['Content-Type: application/json']
buffer = BytesIO()